In [96]:
import numpy as np
import pandas as pd
import requests
from bs4 import BeautifulSoup
import json


base_url= 'https://understat.com/league'
leagues = ['La_liga', 'EPL', 'Bundesliga', 'Serie_A', 'Ligue_1', 'RFPL']
seasons = ['2014', '2015', '2016', '2017', '2018', '2019', '2020', '2021']

url = base_url+'/'+leagues[0]+'/'+seasons[4]
res = requests.get(url)
soup = BeautifulSoup(res.content, "lxml")

scripts = soup.find_all('script')


a=''
for el in  scripts:
    if "teamsData" in str(el):
        a=str(el).strip()
        
ind_start = a.index("('")+2
ind_end = a.index("')")
json_data = a[ind_start:ind_end]

json_data = json_data.encode('utf8').decode('unicode_escape')
data=json.loads(json_data)
teams = {}
for id in data.keys():
    teams[id] = data[id]['title']

columns = []
# Check the sample of values per each column
values = []
for id in data.keys():
    columns = list(data[id]['history'][0].keys())
    values = list(data[id]['history'][0].values())
    break


dataframes = {}
for id, team in teams.items():
    teams_data = []
    for row in data[id]['history']:
         teams_data.append(list(row.values()))
    
    df = pd.DataFrame(teams_data, columns=columns)
    dataframes[team] = df
    
for team, df in dataframes.items():
    dataframes[team]['ppda_coef'] = dataframes[team]['ppda'].apply(lambda x: x['att']/x['def'] if x['def'] != 0 else 0)
    dataframes[team]['oppda_coef'] = dataframes[team]['ppda_allowed'].apply(lambda x: x['att']/x['def'] if x['def'] != 0 else 0)
    
cols_to_sum = ['xG', 'xGA', 'npxG', 'npxGA', 'deep', 'deep_allowed', 'scored', 'missed', 'xpts', 'wins', 'draws', 'loses', 'pts', 'npxGD']
cols_to_mean = ['ppda_coef', 'oppda_coef']

frames = []
for team, df in dataframes.items():
    sum_data = pd.DataFrame(df[cols_to_sum].sum()).transpose()
    mean_data = pd.DataFrame(df[cols_to_mean].mean()).transpose()
    final_df = sum_data.join(mean_data)
    final_df['team'] = team
    final_df['matches'] = len(df)
    frames.append(final_df)
full_stat = pd.concat(frames)

full_stat = full_stat[['team', 'matches', 'wins', 'draws', 'loses', 'scored', 'missed', 'pts', 'xG', 'npxG', 'xGA', 'npxGA', 'npxGD', 'ppda_coef', 'oppda_coef', 'deep', 'deep_allowed', 'xpts']]
full_stat.sort_values('pts', ascending=False, inplace=True)
full_stat.reset_index(inplace=True, drop=True)
full_stat['position'] = range(1,len(full_stat)+1)
full_stat['xG_diff'] = full_stat['xG'] - full_stat['scored']
full_stat['xGA_diff'] = full_stat['xGA'] - full_stat['missed']
full_stat['xpts_diff'] = full_stat['xpts'] - full_stat['pts']
cols_to_int = ['wins', 'draws', 'loses', 'scored', 'missed', 'pts', 'deep', 'deep_allowed']
full_stat[cols_to_int] = full_stat[cols_to_int].astype(int)

col_order = ['position','team', 'matches', 'wins', 'draws', 'loses', 'scored', 'missed', 'pts', 'xG', 'xG_diff', 'npxG', 'xGA', 'xGA_diff', 'npxGA', 'npxGD', 'ppda_coef', 'oppda_coef', 'deep', 'deep_allowed', 'xpts', 'xpts_diff']
full_stat = full_stat[col_order]
pd.options.display.float_format = '{:,.2f}'.format
full_stat.head(10)

,position,team,matches,wins,draws,loses,scored,missed,pts,xG,...,xGA,xGA_diff,npxGA,npxGD,ppda_coef,oppda_coef,deep,deep_allowed,xpts,xpts_diff
0,1,Barcelona,38,26,9,3,90,36,87,83.28,...,44.93,8.93,43.44,33.14,9.02,16.40,417,171,73.96,-13.04
1,2,Atletico Madrid,38,22,10,6,55,29,76,51.87,...,41.43,12.43,37.72,11.01,11.07,11.10,252,190,59.43,-16.57
2,3,Real Madrid,38,21,5,12,63,46,68,68.65,...,48.68,2.68,42.73,19.24,8.90,14.78,341,168,64.77,-3.23
3,4,Valencia,38,15,16,7,51,35,61,61.88,...,42.85,7.85,36.91,19.66,12.96,9.47,278,215,65.16,4.16
4,5,Sevilla,38,17,8,13,62,47,59,69.16,...,46.71,-0.29,41.51,23.03,10.65,10.02,321,211,65.08,6.08
5,6,Getafe,38,15,14,9,48,35,59,47.03,...,44.23,9.23,39.02,3.56,8.77,5.70,186,196,53.19,-5.81
6,7,Espanyol,38,14,11,13,48,50,53,50.16,...,54.62,4.62,48.55,-1.36,9.86,9.82,241,241,50.09,-2.91
7,8,Athletic Club,38,13,14,11,41,45,53,44.44,...,47.16,2.16,43.44,-4.53,8.30,11.30,221,185,50.01,-2.99
8,9,Real Sociedad,38,13,11,14,45,46,50,47.99,...,48.09,2.09,45.68,-5.13,9.94,9.49,194,208,51.13,1.13
9,10,Alaves,38,13,11,14,39,50,50,40.87,...,54.53,4.53,50.07,-11.43,11.23,7.10,129,270,44.02,-5.98


In [90]:
dataframes['Barcelona'].head(2)

,h_a,xG,xGA,npxG,npxGA,ppda,ppda_allowed,deep,deep_allowed,scored,...,xpts,result,date,wins,draws,loses,pts,npxGD,ppda_coef,oppda_coef
0,h,3.26753,0.248353,3.26753,0.248353,"{'att': 118, 'def': 17}","{'att': 407, 'def': 13}",20,0,3,...,2.9009,w,2018-08-18 23:15:00,1,0,0,3,3.019177,6.941176,31.307692
1,a,1.20392,0.510742,1.20392,0.510742,"{'att': 163, 'def': 16}","{'att': 316, 'def': 15}",15,4,1,...,1.9865,w,2018-08-25 23:15:00,1,0,0,3,0.693178,10.187500,21.066667


In [74]:
string = ' xoxo love xoxo ' # Leading and trailing whitespaces are removed print(string.strip())
string.strip()

'xoxo love xoxo'